## Gaurdrails AI Notebook
https://www.guardrailsai.com/docs

In [1]:
## en_core_web_lg is spaCy’s large-size English model

!python -m spacy download en_core_web_lg -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
## Igroning warnings

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="guardrails")

In [3]:
from guardrails import Guard
from pydantic import BaseModel
from typing import List

#### Gaurd Validation Scenario
Validate any LLM output against the MovieReview schema. This gaurd will check if the output is a valid movie review JSON object with all the required fields.

In [4]:
class MovieReview(BaseModel):
    title: str
    sentiment: str  # 'positive' or 'negative'
    key_points: List[str]

In [5]:
gaurd = Guard.for_pydantic(output_class=MovieReview)

In [6]:
gaurd

Guard(id='DBQIYC', name='gr-DBQIYC', description=None, validators=[], output_schema=ModelSchema(definitions=None, dependencies=None, anchor=None, ref=None, dynamic_ref=None, dynamic_anchor=None, vocabulary=None, comment=None, defs=None, prefix_items=None, items=None, contains=None, additional_properties=None, properties={'title': {'title': 'Title', 'type': 'string'}, 'sentiment': {'title': 'Sentiment', 'type': 'string'}, 'key_points': {'items': {'type': 'string'}, 'title': 'Key Points', 'type': 'array'}}, pattern_properties=None, dependent_schemas=None, property_names=None, var_if=None, then=None, var_else=None, all_of=None, any_of=None, one_of=None, var_not=None, unevaluated_items=None, unevaluated_properties=None, multiple_of=None, maximum=None, exclusive_maximum=None, minimum=None, exclusive_minimum=None, max_length=None, min_length=None, pattern=None, max_items=None, min_items=None, unique_items=None, max_contains=None, min_contains=None, max_properties=None, min_properties=None, r

In [7]:
# Sample output from LLM (just for simulation purpose)

raw_output = """
{
  "title": "Avengers Endgame",
  "sentiment": "positive",
  "key_points": ["Mind-bending plot", "Brilliant direction"]
}
"""

In [8]:
validated_output = gaurd.parse(raw_output)
validated_output

/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


ValidationOutcome(call_id='5106178064', raw_llm_output='\n{\n  "title": "Avengers Endgame",\n  "sentiment": "positive",\n  "key_points": ["Mind-bending plot", "Brilliant direction"]\n}\n', validation_summaries=[], validated_output={'title': 'Avengers Endgame', 'sentiment': 'positive', 'key_points': ['Mind-bending plot', 'Brilliant direction']}, reask=None, validation_passed=True, error=None)

In [9]:
## Check the validation
def check_validation(validated_output):
    if validated_output.validation_passed:
        print("Validation Passed!")
        print(validated_output.validated_output)
    else:
        print("Validation Failed!")
        print("Reason:", validated_output.reask.fail_results[0].error_message)

In [10]:
check_validation(validated_output)

Validation Passed!
{'title': 'Avengers Endgame', 'sentiment': 'positive', 'key_points': ['Mind-bending plot', 'Brilliant direction']}


In [11]:
## Another sample output from LLM (just for simulation purpose)
raw_output = '''
{
  "title": "Inception",
  "key_points": ["Mind-bending plot", "Brilliant direction"]
}
'''

In [12]:
failed_output = gaurd.parse(raw_output)
check_validation(failed_output)

Validation Failed!
Reason: JSON does not match schema:
{
  "$": [
    "'sentiment' is a required property"
  ]
}


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# ------------------------------
# Ask the LLM to generate structured JSON output
# ------------------------------
prompt = """
Generate a structured JSON response for a movie review with the following keys:
- title: name of the movie
- sentiment: 'positive' or 'negative'
- key_points: a list of 2–3 bullet points summarizing the movie

Movie: Avengers Endgame
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that always responds in valid JSON only."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.3
)

# ------------------------------
# Print the generated structured JSON
# ------------------------------
generated_output = response.choices[0].message.content
print(generated_output)


```json
{
  "title": "Avengers Endgame",
  "sentiment": "positive",
  "key_points": [
    "Epic conclusion to the Marvel Cinematic Universe's Infinity Saga with a satisfying resolution.",
    "Outstanding performances by the cast, particularly Robert Downey Jr. and Chris Evans.",
    "Visually stunning action sequences and emotional depth that resonate with audiences."
  ]
}
```


In [19]:
validate_movie_output = gaurd.parse(generated_output)
check_validation(validate_movie_output)

Validation Passed!
{'title': 'Avengers Endgame', 'sentiment': 'positive', 'key_points': ["Epic conclusion to the Marvel Cinematic Universe's Infinity Saga with a satisfying resolution.", 'Impressive ensemble cast performances, bringing together numerous beloved characters.', 'Stunning visual effects and action sequences that elevate the storytelling experience.']}


/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
